In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from mlxtend.preprocessing import minmax_scaling 
import tensorflow as tf
from tensorflow import keras
import math
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.metrics import accuracy_score, f1_score
from tensorflow import squeeze


import configparser
config = configparser.ConfigParser()
config.read('config.ini')

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

import wandb
from wandb.keras import WandbCallback
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


2.9.2
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


True

In [9]:
df = pd.read_csv('Data/cleaned.csv', index_col=0)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,102.5,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,32.0,169.5,30.1,0.349,47,1


In [10]:
scaled_data = minmax_scaling(df,columns=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age'])
scaled_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.352941,0.670968,0.489796,0.304348,0.186899,0.314928,0.234415,0.483333
1,0.058824,0.264516,0.428571,0.239130,0.106370,0.171779,0.116567,0.166667
2,0.470588,0.896774,0.408163,0.271739,0.186899,0.104294,0.253629,0.183333
3,0.058824,0.290323,0.428571,0.173913,0.096154,0.202454,0.038002,0.000000
4,0.000000,0.600000,0.163265,0.304348,0.185096,0.509202,0.943638,0.200000
...,...,...,...,...,...,...,...,...
763,0.588235,0.367742,0.530612,0.445652,0.199519,0.300613,0.039710,0.700000
764,0.117647,0.503226,0.469388,0.217391,0.106370,0.380368,0.111870,0.100000
765,0.294118,0.496774,0.489796,0.173913,0.117788,0.163599,0.071307,0.150000
766,0.058824,0.529032,0.367347,0.271739,0.186899,0.243354,0.115713,0.433333


In [11]:
X, y = df.drop('Outcome', axis=1).values, df.Outcome.values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True, random_state=42)

X_train = tf.cast(tf.constant(X_train), tf.float32)
X_test = tf.cast(tf.constant(X_test), tf.float32)
y_train = tf.cast(tf.expand_dims(y_train, axis=1), tf.float32)
y_test = tf.cast(tf.expand_dims(y_test, axis=1), tf.float32)

In [12]:
def build_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=[len(scaled_data.keys())]),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

    optimizer = tf.keras.optimizers.RMSprop(0.01)

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = build_model()

run = wandb.init(project="basic-nn-tensorflow", id="vivid-serenity-1337")
wandb.config.update(config)

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 72        
                                                                 
 dense_4 (Dense)             (None, 4)                 36        
                                                                 
 dense_5 (Dense)             (None, 1)                 5         
                                                                 
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


In [13]:
EPOCHS = 1000

history = model.fit(X_train, y_train, X_ epochs=config['default'].getint('epochs'), verbose=2, callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/500


2023-03-28 12:18:07.692505: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-28 12:18:07.988586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


20/20 - 1s - loss: 2.7229 - accuracy: 0.5635 - 839ms/epoch - 42ms/step
Epoch 2/500
20/20 - 0s - loss: 0.6894 - accuracy: 0.6466 - 164ms/epoch - 8ms/step
Epoch 3/500
20/20 - 0s - loss: 0.6564 - accuracy: 0.6498 - 154ms/epoch - 8ms/step
Epoch 4/500
20/20 - 0s - loss: 0.6504 - accuracy: 0.6531 - 161ms/epoch - 8ms/step
Epoch 5/500
20/20 - 0s - loss: 0.6473 - accuracy: 0.6531 - 179ms/epoch - 9ms/step
Epoch 6/500
20/20 - 0s - loss: 0.6467 - accuracy: 0.6531 - 153ms/epoch - 8ms/step
Epoch 7/500
20/20 - 0s - loss: 0.6460 - accuracy: 0.6531 - 168ms/epoch - 8ms/step
Epoch 8/500
20/20 - 0s - loss: 0.6459 - accuracy: 0.6531 - 182ms/epoch - 9ms/step
Epoch 9/500
20/20 - 0s - loss: 0.6459 - accuracy: 0.6531 - 174ms/epoch - 9ms/step
Epoch 10/500
20/20 - 0s - loss: 0.6460 - accuracy: 0.6531 - 177ms/epoch - 9ms/step
Epoch 11/500
20/20 - 0s - loss: 0.6459 - accuracy: 0.6531 - 177ms/epoch - 9ms/step
Epoch 12/500
20/20 - 0s - loss: 0.6458 - accuracy: 0.6531 - 198ms/epoch - 10ms/step
Epoch 13/500
20/20 - 0s

In [14]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,epoch
495,0.645645,0.653094,495
496,0.645958,0.653094,496
497,0.645947,0.653094,497
498,0.645875,0.653094,498
499,0.645742,0.653094,499


In [15]:
acc = (hist['accuracy'].tail().sum())*100/5 
val_acc = (hist['val_accuracy'].tail().sum())*100/5 

print("Training Accuracy = {}% and Validation Accuracy = {}%".format(round(acc, 2), round(val_acc, 2)))

KeyError: 'val_accuracy'

In [ ]:
tf.keras.models.save_model(model.state_dict(), "models/model_tf.pth")
wandb.save('runs/pima_run_2023-03-28-tf')
wandb.finish()